## SEP and f105w

import matplotlib, numpy, astropy.io, and sep

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import sep

read in and display the image data

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"

In [ ]:
image_data = fits.getdata(fname)
print(type(image_data))
print(image_data.shape)

because the image.fits is in a non-native byte order, we have to change the byte order of the array so SEP and astropy can use the image

In [ ]:
data = image_data.byteswap(False).newbyteorder()

plot the initial image data, using the mean and standard deviation as boundaries on the graph

- note: remember to change "image_data" to "data" after we changed the byte order

In [ ]:
m = np.mean(data)
s = np.std(data)
plt.imshow(data, interpolation='nearest', cmap='inferno', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

### Background Subtraction

measure the "spatially varying background" of the image, aka filter out and show what is in the background

In [ ]:
bkg = sep.Background(data)

show the "global mean" (back) and the noise (rms)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

##### globalback

read the background as a 2-d array, like the initial image (fits.getdata), and plot

In [ ]:
bkg_image = bkg.back()

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='inferno', origin='lower')
plt.colorbar();

##### globalrms

repeat the steps from globalback for rms

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='inferno', origin='lower')
plt.colorbar();

subtract the background from the original image, so it's easier to find objects

In [ ]:
data_sub = data - bkg

### Object Detection

run object detection on the background subtracted image

In [ ]:
objects = sep.extract(data_sub, 15, err=bkg.globalrms)

find how many objects were found in the image

In [ ]:
len(objects)

import Ellipse to circle where the objects are in the image and plot the final image

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m = np.mean(data_sub)
s = np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s,
               origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]),
               width=6*objects['a'][i], height=6*objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    

### Aperture Photometry

- circular aperture photometry with a 3 pixel radius for each object (the first 10)

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0,
                                    err=bkg.globalrms, gain=1.0)

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

### Histogram

histogram the fluxes of each object

In [ ]:
n_bins = 2000
x = flux, fluxerr, flag

In [ ]:
fig = plt.figure(figsize=(12,10))
y_hist, x_hist, ignored = plt.hist(x, bins=n_bins, range=[-5,5], density=True)
xx = np.linspace(-5.0,5.0,1000)
plt.ylim([0,5])
plt.xlim([0,5])
plt.gca().set_aspect(.25)
plt.xlabel('x')
plt.ylabel('y(x)')

plt.show()

save images as PNGs

In [130]:
plt.imsave("f105w_initial.png", data, cmap='gray',  vmin=m-s, vmax=m+s, origin='lower')
plt.imsave("f105w_bkg.png", bkg_image, cmap='gray', origin='lower')
plt.imsave("f105w_rms.png", bkg_rms, cmap='gray', origin='lower')
plt.imsave("f105w_final.png", data_sub, cmap='gray', vmin=m-s, vmax=m+s, origin='lower')